In [1]:
#split current twitter data into sentences with spacy; then analyze sentiment at sentence level
#then apply the weight towards the particular candidate in the sentence, and combine any other candidate sentiment
#output the one with the higher weight

#ex. "Trump my be a liar. But, Crooked Hillary is a terrorist" Trump=-.6 Hillary=-.7 so Output: Hillary Negative

In [3]:
import sys
import csv
import pandas as pd
import nltk
import spacy
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
from spacy.tokens import Doc
import numpy
import en_core_web_sm
import re
import vaderSentiment
from gensim.models import KeyedVectors
import time as time
import gc
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from keras.preprocessing.sequence import pad_sequences 
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
sid = SentimentIntensityAnalyzer()
df = pd.read_pickle("text.pkl")
nlp = en_core_web_sm.load()

Using TensorFlow backend.


In [158]:
dfShort=df[:1000000]


[RT, The, DNC, speeches, have, been, about, TRUMP, TRUMP, TRUMP, because, nobody, has, nothing, great, to, say, about, Crooked, Hillary]
RT @DiamondandSilk: The DNC speeches have been about TRUMP TRUMP TRUMP because nobody has nothing great to say about Crooked Hillary. https…
asf asfka asfdas afs
['asfs', 'afdjoe', 'idpew']


In [143]:
text = "? 1 https://adf blah ~ @Trump"

#doc = nlp(text)
doc = nlp(getattr(row, "text"))

def keep_token(tok):
    # This is only an example rule
    removeToken = ['http','@', '%', 'RT', '#', '?']
    keepToken= True
    for removeWord in removeToken:
        if removeWord in tok.text:
            keepToken = False
        #if removeWord in token.text:
        #    return 1
    if tok.pos_  not in {'PUNCT', 'NUM', 'SYM'} and keepToken:
        return tok.text


final_tokens =[]
for token in doc:
    if keep_token(token) != None:
        final_tokens.append(keep_token(token))


In [136]:
final_tokens

['RT',
 'The',
 'DNC',
 'speeches',
 'have',
 'been',
 'about',
 'TRUMP',
 'TRUMP',
 'TRUMP',
 'because',
 'nobody',
 'has',
 'nothing',
 'great',
 'to',
 'say',
 'about',
 'Crooked',
 'Hillary']

In [162]:
#DON'T NEED TO RUN
#gets text with just words
input_data=[]
docCount=0
tempTime =time.time()
gc.disable()
for row in dfShort.itertuples(index=True, name='Pandas'):
    try:
        doc = nlp(getattr(row, "text"))
        line=[]
        for token in doc:
            if keep_token(token) != None:
                line.append(keep_token(token))

            #print(sent.text)

        input_data.append(line)
    except:
        nlp = en_core_web_sm.load()
        
    docCount=docCount+1
    if(docCount%25000==0):
        print(time.time()-tempTime)
        tempTime= time.time()
        print (docCount)

358.0124433040619
25000
379.1131820678711
50000
483.4844105243683
75000
587.1508343219757
100000
533.9702923297882
125000
513.1495654582977
150000
823.4513041973114
175000
708.4676692485809
200000
497.4711627960205
225000
367.9246435165405
250000
371.5913772583008
275000
357.9151430130005
300000
345.71323347091675
325000
343.6008393764496
350000
343.90509390830994
375000
378.9089422225952
400000
381.77172470092773
425000
371.8495943546295
450000
330.722599029541
475000
355.64640498161316
500000
332.06474471092224
525000
388.62852334976196
550000
388.1786985397339
575000
376.65934681892395
600000
347.62042474746704
625000
358.80620312690735
650000
339.1347577571869
675000
351.08659744262695
700000
347.1168773174286
725000
343.843496799469
750000
351.12280321121216
775000
351.02990198135376
800000
616.1588535308838
825000
7088.712604761124
850000
13153.617839336395
875000
13037.634585857391
900000
9252.59896016121
925000
367.4043743610382
950000
362.270476102829
975000
335.4904317855835


In [163]:
#create Model
model = Word2Vec(input_data, 
                 min_count=3,   # Ignore words that appear less than this
                 size=200,      # Dimensionality of word embeddings
                 workers=2,     # Number of processors (parallelisation)
                 window=5,      # Context window for words during training
                 iter=30)   

In [165]:
#convertDF
df = pd.DataFrame(input_data)

In [176]:
#saveDF
df.to_csv("tweetWordVectors.csv")

In [166]:
#save model
model.save("twitterModel_1000000")

In [4]:
#START RUNNING FROM HERE
#load model
model = KeyedVectors.load("twitterModel_1000000")

In [5]:
#check length of model
len(model.wv.vocab)

66715

In [175]:
model.wv.most_similar('Clinton')

[('clinton', 0.4640190005302429),
 ('HRC', 0.40832608938217163),
 ('She', 0.38673704862594604),
 ('Trump', 0.38302820920944214),
 ('she', 0.37729647755622864),
 ('HillaryClinton', 0.36659738421440125),
 ('her', 0.3572655916213989),
 ('Bernie', 0.3433588743209839),
 ('Tim', 0.34238624572753906),
 ('the', 0.3072303533554077)]

In [148]:
# For example, 'state_of_affairs' will be detected because 'of' is provided here: 
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
# Create the relevant phrases from the list of sentences:
phrases = Phrases(input_data, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)

# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[input_data])

In [149]:
all_sentences

[['So',
  'just',
  'like',
  'every',
  'crazy',
  'fucking',
  'thing',
  'spewing',
  'from',
  'his',
  'pie',
  'hole',
  'Trump',
  'Russia_hacking',
  'comments',
  'were',
  'sarcastic'],
 ['Because',
  'HRC',
  'has',
  'been',
  'slandered',
  'by',
  'misogynistic',
  'press',
  'for',
  'yrs',
  'who',
  'have',
  'Trump',
  'free',
  'air',
  'time'],
 ['Police_shot',
  'and',
  'killed_on',
  'duty_during',
  'term_presidencies',
  'Reagan_576Clinton',
  'II'],
 ['Please',
  'look',
  'into',
  'this',
  'election',
  'deeply',
  'and',
  'with',
  'your',
  'own',
  'eyes',
  "Do_n't",
  'follow',
  'what',
  'your',
  'family',
  'thinks',
  'or',
  'your',
  'friends',
  'READ',
  'U'],
 ['LA',
  'Times',
  'USC',
  'NATIONAL',
  'POLL',
  'TRUMP',
  'CrookedHillary',
  'CELfzYtjwL'],
 ['Tim_Kaine',
  'mails',
  'in',
  'the',
  'warranty',
  'card',
  'on',
  'his',
  'wifes',
  'vibrator'],
 ['Will',
  'outline',
  'his',
  'support',
  'for',
  'the',
  'Muslim',
  